In [1]:
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
import numpy as np
import copy

import os
import sys
import pathlib
pth=str(pathlib.Path().absolute())

#Utilities
sys.path.append(pth)
from model_An import *
from util import *
from Data_loader_image import *

#Base de datos
#sys.path.append()
#data=("/").join(pth.split("/")[:-2])+"/Data_Base/Metada_V6G_p1"
data=("/").join(pth.split("/")[:-2])+"/Data_Base/Metada_V6G"
data_arg=("/").join(pth.split("/")[:-2])+"/Data_Base"


In [3]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [2]:
#dataset=Rotated_Dataset(data_arg,"Metadata_V6G_p1")
dataset=Rotated_Dataset(data_arg,"Metadata_V6G")

In [12]:
train_set, val_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.7), len(dataset)-int(len(dataset)*0.7)])

In [3]:
def train_model(
    dataset,
        epochs,
        batch_size,
        use_cuda,
    folds=5,
        disable_tqdm=False,
        ):
    print("Reading dataset")
    #dset = MNIST(dset_folder,download=True)
    #DATASET
    #imgs = dset.data.unsqueeze(-1).numpy().astype(np.float64)
    #labels = dset.targets.numpy()
    
    #train_idx, valid_idx = map(np.array,util.split_dataset(labels))
    ind=np.arange(0,len(dataset))
    
    #INSIDE K-FOLD
    last_results={}
    results={}
    for fold in range(folds):
        #generate train-test
        print("fold"+str(fold))
        
        i=fold

        ind=np.arange(0,len(dataset))
        indexes=ind[np.random.permutation(len(dataset))]
        L=len(dataset)
        test_idx=indexes[int(i*L/5):int((i+1)*L/5)]
        train_idx=np.delete(indexes,np.arange(int(i*L/5),int((i+1)*L/5)))
        
        model = GAT_ANE_(41,1)
        if use_cuda:
            model = model.cuda()
    
        opt = torch.optim.Adam(model.parameters())
    
        best_valid_acc = 0.
        best_model = copy.deepcopy(model)
    
        last_epoch_train_loss = 0.
        last_epoch_train_acc = 0.
        last_epoch_valid_acc = 0.
    
        interrupted = False
        
        train_dat=np.vectorize(lambda ind:dataset[ind],otypes=[object])(train_idx)
        train_graph=np.vectorize(lambda b:b["image_graph"])(train_dat)
        train_label=np.vectorize(lambda b:b["landmarks"])(train_dat)
        test_dat=np.vectorize(lambda ind:dataset[ind],otypes=[object])(test_idx)
        test_graph=np.vectorize(lambda b:b["image_graph"])(test_dat)
        test_label=np.vectorize(lambda b:b["landmarks"])(test_dat)
        
        loss_function = nn.MSELoss()
        for e in tqdm(range(epochs), total=epochs, desc="Epoch ", disable=disable_tqdm,):
            try:
                #train_losses, train_accs = train_(model, opt, train_graph, train_label,loss_function,
                                                  #batch_size=batch_size, use_cuda=use_cuda, disable_tqdm=disable_tqdm,)
                train_losses, train_accs =train_(model=model,
                                                 optimiser=opt,
                                                 graphs=train_graph,
                                                 labels=train_label,
                                                 use_cuda=use_cuda,
                                                 loss_function=nn.MSELoss(),
                                                 #batch_size=1,
                                                 batch_size=batch_size,
                                                 disable_tqdm=disable_tqdm,
                                                 profile=False)
            
                last_epoch_train_loss = np.mean(train_losses)
                last_epoch_train_acc = 100*np.mean(train_accs)
            except KeyboardInterrupt:
                print("Training interrupted!")
                interrupted = True
        
            #valid_accs = test_(model,test_graph,test_label,use_cuda,desc="Validation ", disable_tqdm=disable_tqdm,)
            valid_accs = test_(model=model,
                               graphs=test_graph,
                               labels=test_label,
                               use_cuda=use_cuda,
                               batch_size=int(32),
                               desc="Test ",
                               disable_tqdm=False)
                
            last_epoch_valid_acc = 100*np.mean(valid_accs)
        
            if last_epoch_valid_acc>best_valid_acc:
                best_valid_acc = last_epoch_valid_acc
                best_model = copy.deepcopy(model)
        
            tqdm.write("EPOCH SUMMARY {loss:.4f} {t_acc:.2f}% {v_acc:.2f}%".format(loss=last_epoch_train_loss, t_acc=last_epoch_train_acc, v_acc=last_epoch_valid_acc))
        
            if interrupted:
                break
    
        last_results[fold]={"train_acc":train_accs,"train_loss":train_losses,"valid_acc":valid_accs}
        results[fold]={"train_acc":last_epoch_train_acc,"train_loss":last_epoch_train_loss,"valid_acc":last_epoch_valid_acc}
        save_model("best"+str(fold),best_model)
        save_model("last"+str(fold),model)
    np.save("results"+'.npy',results)
    return results
        


In [4]:
results=train_model(
    dataset,
    epochs=35,
    batch_size=int(100),
    use_cuda=True,
    folds=5,
        disable_tqdm=False,
        )

Reading dataset
fold0


Epoch :   0%|          | 0/35 [00:00<?, ?it/s]
                                               ?it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]       
                                               ?it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]       
                                               ?it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]       
Instances :   0%|          | 0/37.53 [00:00<?, ?it/s]
                                              :01, 21.97it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 21.97it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 21.97it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
Instances :   8%|▊         | 3/37.53 [00:00<00:01, 21.97it/s]
Instances :  16%|█▌        | 6/37.53 [00:00<00:01, 25.38it/s]

135.1180	13511.80%	[0.13461207] (x[50])
146.0604	14606.04%	[0.15637341] (x[91])
144.5466	14454.66%	[0.17826319] (x[91])
147.3904	14739.04%	[0.20029804] (x[83])
137.6414	13764.14%	[0.22252977] (x[79])
141.5750	14157.50%	[0.2449807] (x[68])


                                              
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 25.38it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 25.38it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
Instances :  16%|█▌        | 6/37.53 [00:00<00:01, 25.38it/s]
                                              :01, 26.06it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 26.06it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
                                              :01, 26.06it/s]
Epoch :   0%|          | 0/35 [00:00<?, ?it/s]               
Instances :  24%|██▍       | 9/37.53 [00:00<00:01, 26.06it/s]
Instances :  32%|███▏      | 12/37.53 [00:00<00:00, 26.40it/s]

141.7546	14175.46%	[0.2674822] (x[92])
138.7556	13875.56%	[0.28986356] (x[67])
143.0103	14301.03%	[0.31224033] (x[59])
142.7552	14275.52%	[0.33523956] (x[52])
135.7209	13572.09%	[0.35872382] (x[72])
140.8590	14085.90%	[0.3818113] (x[99])


...

3.7950	379.50%	[11.852837] (x[64])
2.6688	266.88%	[11.770849] (x[41])
1.4406	144.06%	[11.741087] (x[86])
3.9837	398.37%	[11.785362] (x[99])
2.5533	255.33%	[11.874687] (x[80])
3.2068	320.68%	[11.998249] (x[59])



                                                              
Epoch :  97%|█████████▋| 34/35 [00:57<00:01,  1.63s/it]04it/s]
Instances :  88%|████████▊ | 33/37.52 [00:01<00:00, 26.24it/s]
                                                              
Epoch :  97%|█████████▋| 34/35 [00:57<00:01,  1.63s/it]24it/s]
                                                              
Epoch :  97%|█████████▋| 34/35 [00:57<00:01,  1.63s/it]24it/s]
                                                              
Epoch :  97%|█████████▋| 34/35 [00:57<00:01,  1.63s/it]24it/s]
Instances :  96%|█████████▌| 36/37.52 [00:01<00:00, 26.40it/s]
                                                              
Epoch :  97%|█████████▋| 34/35 [00:57<00:01,  1.63s/it]40it/s]
                                                              
Instances : 101%|██████████| 38/37.52 [00:01<00:00, 25.43it/s]

Test :   0%|          | 0/29.34375 [00:00<?, ?it/s]

5.7625	576.25%	[12.105957] (x[50])
2.7205	272.05%	[12.148689] (x[72])
2.6869	268.69%	[12.186455] (x[54])
2.7483	274.83%	[12.195834] (x[68])
2.9053	290.53%	[12.201118] (x[90])
4.3962	439.62%	[12.180115] (x[37])



Test :  44%|████▍     | 13/29.34375 [00:00<00:00, 129.77it/s]
Test : 30it [00:00, 142.73it/s]                              
Epoch : 100%|██████████| 35/35 [00:57<00:00,  1.65s/it]

EPOCH SUMMARY 3.5770 357.70% 11.40%


In [13]:
folds=np.array(list(results))
np.vectorize(lambda res,k:res[k]["train_acc"]  res[k]["valid_acc"])

In [5]:
results

{0: {'train_acc': 363.7913622055887,
  'train_loss': 3.6379134184435795,
  'valid_acc': 10.586511344882556},
 1: {'train_acc': 356.5463465337364,
  'train_loss': 3.5654633170679997,
  'valid_acc': 10.607038008548217},
 2: {'train_acc': 370.843078734503,
  'train_loss': 3.7084307482368066,
  'valid_acc': 17.182199685558327},
 3: {'train_acc': 329.73906551311836,
  'train_loss': 3.297390545669355,
  'valid_acc': 12.574435271688108},
 4: {'train_acc': 357.7007481810535,
  'train_loss': 3.5770074129104614,
  'valid_acc': 11.401409549327198}}

In [21]:
np.vstack((np.array(results[0]["train_acc"]).reshape(-1,1),np.array(results[0]["valid_acc"]).reshape(-1,1)))

array([[ 7.06124939],
       [ 3.94556824],
       [ 1.32258804],
       [ 4.0828302 ],
       [ 1.51911102],
       [ 4.07700592],
       [ 3.9284259 ],
       [ 2.91217621],
       [ 1.15789803],
       [ 2.49437317],
       [ 2.74138123],
       [ 8.74155396],
       [ 2.70049072],
       [ 4.3369397 ],
       [ 5.52278076],
       [ 2.40586136],
       [ 8.53570679],
       [ 1.17074036],
       [ 4.09988098],
       [ 1.21529709],
       [ 1.41680679],
       [ 4.2254541 ],
       [ 2.96023529],
       [ 3.99266815],
       [ 1.61227829],
       [ 4.23587616],
       [ 2.69038727],
       [ 2.60479797],
       [ 4.11155365],
       [ 2.85009247],
       [ 5.59345459],
       [ 3.84716339],
       [ 1.2260083 ],
       [ 4.51823578],
       [ 2.59225983],
       [ 4.12614716],
       [ 3.98409973],
       [ 1.48654304],
       [ 0.18733967],
       [ 7.25615609],
       [ 0.0698637 ],
       [ 0.19560692],
       [ 0.19263488],
       [ 0.03649099],
       [ 0.03687666],
       [ 0